In [1]:
from tools.ground_removal import *
from tools.ground_removal_kitti import remove_ground
from tools.raycasting import *
import numpy as np
from sklearn.cluster import DBSCAN
from mayavi import mlab
from tools.visualization import *
from scipy.spatial import KDTree
import yaml
%load_ext autoreload
%autoreload 2

In [13]:
gr = Ground_removal('01')
pts, _ = gr.get_frame(20)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], mode='point')
mlab.show()

In [3]:
gr = Ground_removal('18')
first_frame = 1
second_frame = first_frame + 2
pts, intens = gr.get_synchronized_frames_without_ground(first_frame,1)
pts2, intens2 = gr.get_synchronized_frames_without_ground(second_frame,1)

In [5]:
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], color=(0,1,0), mode='point')
mlab.points3d(pts2[:,0], pts2[:,1], pts2[:,2], color=(0,0,1), mode='point')
mlab.show()

In [39]:
with open('raycast_config.yaml', 'r') as f:
    config = yaml.safe_load(f)
#figure = mlab.figure()
gr = Ground_removal('01')
frame = 240
rp = RaycastPredictor(config['CAR'], gr, verbose=False)
mask = rp.predict(frame)

In [40]:
static = mask == 9
dynamic = mask == 251
unlabeled = mask == 0
print(unlabeled.sum())


11697


In [41]:
pts,_ = gr.get_frame(frame)
mlab.points3d(pts[:,0][dynamic], pts[:,1][dynamic], pts[:,2][dynamic], color=(0,1,0), mode='point')
mlab.points3d(pts[:,0][static], pts[:,1][static], pts[:,2][static], mode='point')
mlab.points3d(pts[:,0][unlabeled], pts[:,1][unlabeled], pts[:,2][unlabeled], color=(1,1,0), mode='point')
mlab.show()

In [200]:
pts,_ = gr.get_frame_without_ground(1000)
clustering = DBSCAN(eps=config['CAR']['EPS'],
                    min_samples=config['CAR']['MIN_SAMPLES']).fit(pts[:,:3])
mask_cluster = clustering.labels_ >= 0
mlab.points3d(pts[:,0][mask_cluster], pts[:,1][mask_cluster],
              pts[:,2][mask_cluster], clustering.labels_[mask_cluster], mode='point')
mlab.show()

In [39]:
# Evaluation
from tools.evaluation import evaluate_kitti
with open('raycast_config.yaml', 'r') as f:
    raycast_config = yaml.safe_load(f)
with open('evaluate_cofig.yaml', 'r') as f:
    evaluation_config = yaml.safe_load(f)

In [8]:
evaluate_kitti(evaluation_config, RaycastPredictor, raycast_config)

class CAR
    sequence: 01
        frame: 000000 -> precision 100.0%, recall 53.8%, iou 53.8%
        frame: 000001 -> precision 100.0%, recall 56.98%, iou 56.98%
        frame: 000002 -> precision 92.31%, recall 59.18%, iou 56.4%
        frame: 000003 -> precision 100.0%, recall 61.46%, iou 61.46%
        frame: 000004 -> precision 100.0%, recall 32.13%, iou 32.13%
        frame: 000005 -> precision 100.0%, recall 26.42%, iou 26.42%
        frame: 000006 -> precision 100.0%, recall 20.45%, iou 20.45%
        frame: 000007 -> precision 100.0%, recall 24.81%, iou 24.81%
        frame: 000390 -> precision 89.74%, recall 60.57%, iou 56.65%
        frame: 000391 -> precision 99.81%, recall 62.5%, iou 62.43%
        frame: 000392 -> precision 99.63%, recall 65.62%, iou 65.46%
        frame: 000393 -> precision 100.0%, recall 72.44%, iou 72.44%
        frame: 000394 -> precision 98.88%, recall 70.72%, iou 70.16%
        frame: 000395 -> precision 99.71%, recall 70.34%, iou 70.2%
        fram

[98.53, 77.44, 76.1]

In [34]:
# closer investigation
gr = Ground_removal('05')
figure = mlab.figure()
rp = RaycastPredictor(raycast_config['PEDESTRIAN'], gr)
#rp.predict(580); pts, intens = gr.get_frame(585)
#mlab.points3d(pts[:,0], pts[:,1], pts[:,2], intens, figure=figure, mode='point')
#plot_prediction_pedestrians(gr, rp, 302, figure)
plot_prediction_cars(gr,rp,603,figure)
mlab.show()

In [45]:
gr = Ground_removal('13')
pts, intens = gr.get_frame_without_ground(2530)
#clustering = DBSCAN(eps=0.2,
#                    min_samples=10).fit(pts[:,:3])
clustering = DBSCAN(eps=raycast_config['PEDESTRIAN']['EPS'],
                    min_samples=raycast_config['PEDESTRIAN']['MIN_SAMPLES']).fit(pts[:,:3])
mask = clustering.labels_ == 19
#mlab.points3d(pts[:,0], pts[:,1], pts[:,2], clustering.labels_, mode='point')
mlab.points3d(pts[:,0][mask], pts[:,1][mask], pts[:,2][mask], color=(1,0,0), mode='point')
mlab.points3d(pts[:,0][~mask], pts[:,1][~mask], pts[:,2][~mask], color=(1,1,1), mode='point')
mlab.show()

In [24]:
gr = Ground_removal('07')
pts, intens = gr.get_synchronized_frames_without_ground(10,5)
#pts, intens = gr.get_frame_without_ground(596)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], pts[:,3], mode='point')
mlab.show()

In [125]:
pts, intens = gr.get_frame_without_ground(601)
pts2, intens = gr.get_frame_without_ground(606)
pts3, _ = gr.get_frame_without_ground(611)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], color=(1,0,0), mode='point')
mlab.points3d(pts2[:,0], pts2[:,1], pts2[:,2], color=(0,1,0), mode='point')
mlab.points3d(pts3[:,0], pts3[:,1], pts3[:,2], color=(0,0,1), mode='point')
mlab.show()

In [119]:
pts, intens = gr.get_synchronized_frames_without_ground(601,5)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], pts[:,3], mode='point')
mlab.show()

In [56]:
cluster_pts = pts[mask]
bb_x_min =  cluster_pts[:,0].min()
bb_x_max =  cluster_pts[:,0].max()
bb_y_min =  cluster_pts[:,1].min()
bb_y_max =  cluster_pts[:,1].max()
bb_z_min =  cluster_pts[:,2].min()
bb_z_max =  cluster_pts[:,2].max()

width = min(abs(bb_x_max - bb_x_min), abs(bb_y_max - bb_y_min))
length = max(abs(bb_x_max - bb_x_min), abs(bb_y_max - bb_y_min))
height = abs(bb_z_max - bb_z_min)
print(width,length, height)

0.6944878394305363 1.4024296786247419 1.6222920092642177


### grid search parameters tuning

In [423]:
from tools.evaluation import grid_search
with open('raycast_config.yaml', 'r') as f:
    raycast_config = yaml.safe_load(f)
with open('evaluate_cofig.yaml', 'r') as f:
    evaluation_config = yaml.safe_load(f)
    
params = {'EPS': [0.4,0.5], 'RADIUS_RAYCAST': [0.2,0.3]}
grid_search(evaluation_config, RaycastPredictor, raycast_config, params)

{'EPS': 0.4, 'MIN_SAMPLES': 10, 'RADIUS_EMPTY_SPACE_CHECK': 1, 'RADIUS_RAYCAST': 0.3, 'MAX_WIDTH': 3, 'MAX_LENGTH': 6, 'MAX_HEIGTH': 2, 'MIN_LENGTH': 1, 'NUM_OF_FRAMES_IN_FUTURE': 2, 'EGO_DELETE_RADIUS': 2.5}
Error, path ../../semantic_kitti_data/sequences/05/velodyne/000578.bin does not exists
Error, path ../../semantic_kitti_data/sequences/05/velodyne/000579.bin does not exists
[95.68, 58.57, 57.93]
{'EPS': 0.5, 'MIN_SAMPLES': 10, 'RADIUS_EMPTY_SPACE_CHECK': 1, 'RADIUS_RAYCAST': 0.3, 'MAX_WIDTH': 3, 'MAX_LENGTH': 6, 'MAX_HEIGTH': 2, 'MIN_LENGTH': 1, 'NUM_OF_FRAMES_IN_FUTURE': 2, 'EGO_DELETE_RADIUS': 2.5}
Error, path ../../semantic_kitti_data/sequences/05/velodyne/000578.bin does not exists
Error, path ../../semantic_kitti_data/sequences/05/velodyne/000579.bin does not exists
[94.29, 60.92, 60.92]
{"{'EPS': 0.4, 'MIN_SAMPLES': 10, 'RADIUS_EMPTY_SPACE_CHECK': 1, 'RADIUS_RAYCAST': 0.3, 'MAX_WIDTH': 3, 'MAX_LENGTH': 6, 'MAX_HEIGTH': 2, 'MIN_LENGTH': 1, 'NUM_OF_FRAMES_IN_FUTURE': 2, 'EGO_